In [1]:
import sentencepiece as spm
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as sps
import nlp
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from src.seq2seq_generator import Seq2Seq, Encoder, Decoder
from src.cnn_discriminator import CNNDiscriminator
from src.utils import *
from src.dataset import Dataset, Padder

import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

sns.set_style('whitegrid')

### 1. load data

In [2]:
dataset_name = 'news'

In [3]:
%%time

if dataset_name == 'cnn':
    train_dataset = nlp.load_dataset("cnn_dailymail", "3.0.0", split="train")
    val_dataset = nlp.load_dataset("cnn_dailymail", "3.0.0", split="validation")
    test_dataset = nlp.load_dataset("cnn_dailymail", "3.0.0", split="test")
    train_articles = [item['article'] for item in train_dataset]
    train_highlights = [item['highlights'] for item in train_dataset]
    val_articles = [item['article'] for item in val_dataset]
    val_highlights = [item['highlights'] for item in val_dataset]
elif dataset_name == 'news':
    news = pd.read_csv('data/news_summary.csv')
    news.headlines = [process_str(s) for s in news.headlines]
    news.text = [process_str(s) for s in news.text]
    X_train, X_test = train_test_split(news, test_size=0.3,
                                       random_state=42)
    train_articles = X_train.text.values
    train_highlights = X_train.headlines.values
    val_articles = X_test.text.values
    val_highlights = X_test.headlines.values

CPU times: user 752 ms, sys: 15.8 ms, total: 767 ms
Wall time: 767 ms


### 2. sentencepiece model

In [4]:
train_new_model = False

In [5]:
%%time
if train_new_model:
    with open('data/news_texts.txt', 'a') as f:
        for article in tqdm(train_articles):
            f.write(article + '\n')
        for highlight in tqdm(train_highlights):
            f.write(highlight + '\n')
        for article in tqdm(val_articles):
            f.write(article + '\n')
        for highlight in tqdm(val_highlights):
            f.write(highlight + '\n')
            
    spm.SentencePieceTrainer.train(input='data/news_texts.txt',
                                   model_prefix='news10k',
                                   vocab_size=10000,
                                   pad_id=0,
                                   bos_id=1,
                                   eos_id=2,
                                   unk_id=3)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


### 3. lda

In [6]:
bigram = False
lemmas = False
train_model = False
modelname = f'topic_models/news_train_bigram{bigram}_lemmas{lemmas}.model'

In [7]:
%%time

articles_train_words = process_texts(train_articles, bigram=bigram, lemmas=lemmas)
highlights_train_words = process_texts(train_highlights, bigram=bigram, lemmas=lemmas)

articles_val_words = process_texts(val_articles, bigram=bigram, lemmas=lemmas)
highlights_val_words = process_texts(val_highlights, bigram=bigram, lemmas=lemmas)

id2word = corpora.Dictionary(articles_train_words)

articles_train_corpus = [id2word.doc2bow(text) for text in articles_train_words]
highlights_train_corpus = [id2word.doc2bow(text) for text in highlights_train_words]

articles_val_corpus = [id2word.doc2bow(text) for text in articles_val_words]
highlights_val_corpus = [id2word.doc2bow(text) for text in highlights_val_words]

CPU times: user 27.2 s, sys: 204 ms, total: 27.4 s
Wall time: 27.4 s


In [8]:
%%time
if train_model:
    lda_model = gensim.models.ldamodel.LdaModel(corpus=articles_train_corpus,
                                                id2word=id2word,
                                                num_topics=10, 
                                                random_state=100,
                                                update_every=1,
                                                chunksize=100,
                                                passes=10,
                                                alpha='auto',
                                                per_word_topics=True)
    lda_model.save(modelname)
else:
    lda_model = gensim.models.ldamodel.LdaModel.load(modelname)

lda_model.print_topics()

CPU times: user 25.9 ms, sys: 8.01 ms, total: 33.9 ms
Wall time: 33 ms


[(0,
  '0.031*"based" + 0.026*"million" + 0.020*"used" + 0.018*"startup" + 0.014*"using" + 0.013*"deal" + 0.012*"public" + 0.011*"worth" + 0.011*"months" + 0.010*"making"'),
 (1,
  '0.030*"khan" + 0.029*"tweeted" + 0.022*"air" + 0.020*"part" + 0.015*"elections" + 0.015*"shah" + 0.015*"upcoming" + 0.015*"user" + 0.014*"flight" + 0.013*"show"'),
 (2,
  '0.028*"india" + 0.022*"also" + 0.018*"years" + 0.013*"accused" + 0.011*"national" + 0.011*"arrested" + 0.011*"actress" + 0.010*"revealed" + 0.010*"leader" + 0.009*"according"'),
 (3,
  '0.081*"said" + 0.033*"added" + 0.019*"film" + 0.011*"would" + 0.010*"like" + 0.009*"actor" + 0.009*"singh" + 0.008*"people" + 0.007*"wrote" + 0.006*"media"'),
 (4,
  '0.063*"us" + 0.040*"president" + 0.024*"modi" + 0.023*"country" + 0.023*"pm" + 0.021*"trump" + 0.017*"pakistan" + 0.015*"narendra" + 0.013*"donald" + 0.011*"house"'),
 (5,
  '0.024*"china" + 0.022*"security" + 0.019*"car" + 0.015*"series" + 0.015*"led" + 0.015*"states" + 0.014*"away" + 0.014*

In [9]:
coherence_model_lda = CoherenceModel(model=lda_model,
                                     texts=articles_train_words,
                                     dictionary=id2word,
                                     coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.38893017640073324


### 5. dataset and dataloaders

In [10]:
vocab_size = 10000
sp_modelname = f'sentencepiece_models/news{int(vocab_size/1000)}k.model'

In [11]:
sp = spm.SentencePieceProcessor(model_file=sp_modelname)

In [12]:
train_dataset = Dataset(train_articles, train_highlights, sp=sp)
val_dataset = Dataset(val_articles, val_highlights, sp=sp)

train_dataloader = DataLoader(train_dataset, batch_size=128,
                              collate_fn=Padder(), shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=128,
                            collate_fn=Padder(), shuffle=False)

In [13]:
for batch in train_dataloader:
    print(batch[0].size())
    print(batch[1].size())
    break

torch.Size([103, 128])
torch.Size([21, 128])


### 6. model architecture

In [14]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

cuda:0


In [15]:
writer = SummaryWriter()

In [16]:
INPUT_DIM = vocab_size
OUTPUT_DIM = vocab_size
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 256
N_LAYERS = 3
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

G = Seq2Seq(enc, dec, device).to(device)
save_path = 'saved_models/gen_adv.pth'
G.load_state_dict(torch.load(save_path))

<All keys matched successfully>

### 8. adversarial training with topic difference reward

In [17]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [18]:
writer = SummaryWriter()

In [19]:
G.train()

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10000, 256)
    (rnn): LSTM(256, 256, num_layers=3, dropout=0.1)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(10000, 256)
    (rnn): LSTM(256, 256, num_layers=3, dropout=0.1)
    (out): Linear(in_features=512, out_features=10000, bias=True)
    (w): Linear(in_features=256, out_features=256, bias=True)
    (attn_lin): Linear(in_features=512, out_features=512, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [20]:
n_epochs = 150
beta = 0.2
teacher_forcing = 0.6

generator_optimizer = torch.optim.Adam(G.parameters(), lr=1e-4)
criterion_ml = nn.CrossEntropyLoss(ignore_index=0)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(generator_optimizer, 'min', patience=10)

G_losses = []
iter_num = -1
G_ce_val_losses = []

In [21]:
generator_optimizer = torch.optim.Adam(G.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=0)

G_losses = []
D_losses = []
iter_num = -1
G_ce_val_losses = []

for epoch_idx in range(n_epochs):
    topic_num = random.choice(np.arange(0, 10))
    topic_token = 8000 + topic_num
    topic_vector = np.zeros(10)
    topic_vector[topic_num] = 1.
    topic_vector = topic_vector.tolist()
    
    train_dataset = Dataset(train_articles, train_highlights, sp=sp, topic_token=topic_token)
    val_dataset = Dataset(val_articles, val_highlights, sp=sp, topic_token=topic_token)

    train_dataloader = DataLoader(train_dataset, batch_size=128,
                                  collate_fn=Padder(), shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=128,
                                collate_fn=Padder(), shuffle=False)
    
    
    beta += 0.02 * epoch_idx
    beta = min(0.95, beta)
    
    teacher_forcing -= 0.02 * epoch_idx
    teacher_forcing = max(0, teacher_forcing)
    
    total_G_loss = 0.
    G.train()
    for batch_idx, data_input in tqdm(enumerate(train_dataloader)):
        iter_num += 1
        article = data_input[0].to(device)
        highlight = data_input[1].to(device)

        # Generator
        generator_optimizer.zero_grad()
        gen_out, generated_highlight = G.sample(article, highlight, teacher_forcing_ratio=teacher_forcing)
        
        texts = tensor_to_text(generated_highlight, sp)
        predicted_corpus = [id2word.doc2bow(text.split()) for text in texts]
        predicted_topics = [get_topic_vector(lda_model, text) for text in predicted_corpus]
        
        # texts = tensor_to_text(article, sp, beam_search=True)
        # article_corpus = [id2word.doc2bow(text.split()) for text in texts]
        # article_topics = [get_topic_vector(lda_model, text) for text in article_corpus]
        
        # gen_list = generated_highlight.cpu().detach().numpy().T.tolist()
        # all_len = generated_highlight.size(1)
        # uniques = np.array(list(map(lambda x: len(np.unique(x)), gen_list)))
        # factors = uniques / all_len
    
        # rewards = [[cosine(topic_vector, predicted_topic)*factors[i]]*generated_highlight.size(0)
        #            for i, predicted_topic in enumerate(predicted_topics)]
        rewards = [[predicted_topic[topic_num]]*generated_highlight.size(0)
                   for predicted_topic in predicted_topics]
        rewards = torch.tensor(rewards).to(device)
        writer.add_scalar('Reward/train', torch.mean(rewards).item(), iter_num)
        # rewards = 1 - rewards
       
        pg_loss = G.batch_pgloss_generated(gen_out, generated_highlight.permute(1, 0), rewards)
        
        out = G(article, highlight, teacher_forcing_ratio=teacher_forcing)
        ml_loss = criterion_ml(out.permute(1, 2, 0), highlight.permute(1, 0))
        
        loss = beta*pg_loss + (1-beta)*ml_loss
        loss.backward()
        
        generator_optimizer.step()
        writer.add_scalar('Loss_train/loss', loss.data.item(), iter_num)
        writer.add_scalar('Loss_train/ml_loss', ml_loss.data.item(), iter_num)
        writer.add_scalar('Loss_train/pg_loss', pg_loss.data.item(), iter_num)
        total_G_loss += loss.data.item() * article.size(1)
        
    G_losses.append(total_G_loss / len(train_dataset))
    print(f'epoch {epoch_idx} G loss: {G_losses[-1]}')
    
    G.eval()
    total_loss = 0.
    total_reward = 0.
    total_ml_loss = 0.
    total_pg_loss = 0.
    for batch_idx, data_input in tqdm(enumerate(val_dataloader), position=0, leave=True):
        article = data_input[0].to(device)
        highlight = data_input[1].to(device)
        gen_out, generated_highlight = G.sample(article, highlight, teacher_forcing_ratio=0.)
        
        texts = tensor_to_text(generated_highlight, sp)
        predicted_corpus = [id2word.doc2bow(text.split()) for text in texts]
        predicted_topics = [get_topic_vector(lda_model, text) for text in predicted_corpus]
        
        # texts = tensor_to_text(article, sp)
        # article_corpus = [id2word.doc2bow(text.split()) for text in texts]
        # article_topics = [get_topic_vector(lda_model, text) for text in article_corpus]
    
        # rewards = [[cosine(topic_vector, predicted_topic)]*generated_highlight.size(0)
        #            for predicted_topic in predicted_topics]
        rewards = [[predicted_topic[topic_num]]*generated_highlight.size(0)
                   for predicted_topic in predicted_topics]
        rewards = torch.tensor(rewards).to(device)
        total_reward += torch.mean(rewards).item() * article.size(1)
        # rewards = 1 - rewards
        pg_loss = G.batch_pgloss_generated(gen_out, generated_highlight.permute(1, 0), rewards)
        
        out = G(article, highlight, teacher_forcing_ratio=0.)
        ml_loss = criterion_ml(out.permute(1, 2, 0), highlight.permute(1, 0))
        
        loss = beta*pg_loss + (1-beta)*ml_loss
        total_loss += loss.data.item() * article.size(1)
        
        total_ml_loss += ml_loss.data.item() * article.size(1)
        total_pg_loss += pg_loss.data.item() * article.size(1)
        
    writer.add_scalar('Reward/val', total_reward / len(val_dataset), iter_num)
    writer.add_scalar('Loss_val/loss', total_loss / len(val_dataset), iter_num)
    writer.add_scalar('Loss_val/ml_loss', total_ml_loss / len(val_dataset), iter_num)
    writer.add_scalar('Loss_val/pg_loss', total_pg_loss / len(val_dataset), iter_num)
    
    writer.add_scalar('Params/beta', beta, iter_num)
    writer.add_scalar('Params/teacher_forcing', teacher_forcing, iter_num)
    
    scheduler.step(total_loss)
    
    indices = sps.randint(0, out.size(1)).rvs(size=10)
    pred_texts = tensor_to_text(out[:, indices, :], sp, beam_search=True)
    truth_texts = tensor_to_text(highlight[:, indices], sp)
    for pred, truth in zip(pred_texts, truth_texts):
        writer.add_text('Texts/pred', pred, iter_num)
        writer.add_text('Texts/truth', truth, iter_num)
        print(f'predicted: {pred}')
        print(f'truth: {truth}\n')

539it [02:43,  3.30it/s]
1it [00:00,  8.46it/s]

epoch 0 G loss: 2.2161131079881105


231it [00:29,  7.93it/s]
0it [00:00, ?it/s]

predicted: un probing syrian chief syrians over syrian war
truth: enough evidence to convict syrian prez of war crimes un

predicted: doctor hospital hospital hospital hospital hospitalss dised toss
truth: docs chant mantra for hours to bring down deaths in hospital

predicted: fake news of shah rukh khan death on on news crash crash
truth: fake news of srk s death emerges on european news channel

predicted: fake news of shah rukh khan death on on news crash crash
truth: fake news of srk s death emerges on european news channel

predicted: srikanth sed padma shri shri india s sth civilian
truth: kidambi srikanth conferred with padma shri award

predicted: 36 deported 36 year old engineer over suspected links isis
truth: uae deports indian over suspected links with isis

predicted: farmers defy milk milk at 10 dayss on farmers
truth: farmers dump vegetables spill milk on roads during protest

predicted: 33 inmates married at gun6 in 2016
truth: 3 075 grooms got married in bihar at gunp

539it [02:42,  3.31it/s]
1it [00:00,  8.39it/s]

...

231it [00:29,  7.84it/s]
0it [00:00, ?it/s]

predicted: was tededed flight india flightedededed flight
truth: air india refuses to let wheelchair bound passenger on plane

predicted: 4 missiles fired in taliban pakistan pakistan
truth: 4 missiles fired into pakistan from afghanistan

predicted: us co stream shot dead by by in in
truth: 22 year old model shot dead inside her porsche in iraq

predicted: b luru police police for idiotinginginging with helmet helmet
truth: b luru police ask people to wear helmets share 10 secs challenge

predicted: vajpayee pm vajpayee to to in inhindi at at party at
truth: late pm vajpayee was 1st person to give hindi speech at un

predicted: freshable tost soldiers soldiers soldiers soldiersss army army
truth: soldiers to face action for disrespecting militants bodies

predicted: my my my i am i my me me me meuddinuddin
truth: threat to my life claims aimim leader akbaruddin owaisi

predicted: got the for for for big b am bachchan on s yr yrs of 13
truth: received best compliment from dilip kumar f

539it [02:42,  3.31it/s]
1it [00:00,  7.65it/s]

epoch 144 G loss: 0.5580843387394352


106it [00:13,  7.94it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
save_path = 'saved_models/gen_topics_ohe_3.pth'
torch.save(G.state_dict(), save_path)